In [ ]:
### 0. 임포트
# 데이터 프레임 관련 임포트
import numpy  as np
import pandas as pd

# 웹 관련 임포트
import urllib
import requests
from selenium import webdriver # version_3.14.1
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

# 기타
import time
import psycopg2
import datetime
from tqdm import tqdm
from glob import glob
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', family='Malgun Gothic')

# 경고 처리
import warnings
warnings.filterwarnings('ignore')

### 최종

### 크롤링 test

ver1

In [ ]:
### 1. 필요한 정보 추출
# 1-1. 시도 정보 추출
def getSido():
    global sido_names
    sido_list_selectbox = driver.find_element_by_xpath('//*[@id="ctpvCd"]') # 셀렉트 박스 선택
    sido_list = sido_list_selectbox.find_elements_by_tag_name('option') # 셀렉트 박스 안에 있는 옵션 선택
    sido_names = [sido.text for sido in sido_list] # 시도명 리스트에 담기
    sido_names = sido_names[1:-1]

# 1-2. 선택 지역의 시군구 정보 추출
def getSigungu(sido):
    global sigungu_names
    sigungu_names = []

    # 시도 선택
    element = driver.find_element_by_id('ctpvCd')
    element.send_keys(sido)
    time.sleep(1)

    # 각 시도의 시군구 정보 추출
    sigungu_list_selectbox = driver.find_element_by_xpath('//*[@id="sggCd"]') # 셀렉트 박스 선택
    sigungu_list = sigungu_list_selectbox.find_elements_by_tag_name('option') # 셀렉트 박스 안에 있는 옵션 선택
    time.sleep(1)
    sigungu_name = [sigungu.text for sigungu in sigungu_list] # 시군구명 리스트에 담기
    sigungu_names.extend(sigungu_name)
    sigungu_names = sigungu_names[2:]

# 1-3. id값 으로 버튼 클릭
def clickChecktBox():
    global checked
    checked = []
    num = 0
    driver.switch_to_window(driver.window_handles[1])
    time.sleep(1)
    
    while True:
        try:
            # 조 목록 가져오기
            num += 1
            jo = driver.find_element_by_xpath(f'/html/body/div/div/div/div[2]/div[2]/div[2]/div/span[{num}]').text
            time.sleep(1)

            # 조 목록에 용적률, 건폐율이 포함된 조항만 체크박스 선택
            if '용도지역' in jo and '안' in jo and '용적률' in jo:
                joid = driver.find_element_by_xpath(f'/html/body/div/div/div/div[2]/div[2]/div[2]/div/span[{num}]/label').get_attribute('for') # 해당 조항의 id값 가져오기
                button = driver.find_element_by_id(joid) # id값 선택
                driver.execute_script('arguments[0].click();', button) # 체크박스 클릭
                checked.append(joid)
                time.sleep(1)
            elif '용도지역' in jo and '안' in jo and '건폐율' in jo:
                joid = driver.find_element_by_xpath(f'/html/body/div/div/div/div[2]/div[2]/div[2]/div/span[{num}]/label').get_attribute('for') # 해당 조항의 id값 가져오기
                button = driver.find_element_by_id(joid) # id값 선택
                driver.execute_script('arguments[0].click();', button) # 체크박스 클릭
                checked.append(joid)
                time.sleep(1)
            else:
                pass
        except:
            break

# 1-4. 용적률, 건폐율 파일 다운로드
def downloadFile():
    if len(checked) > 0:
        # txt 저장
        driver.find_element_by_xpath('/html/body/div/div/div/div[1]/a[1]').click()
        time.sleep(4)

        # 창 닫기, 검색 창으로 전환
        driver.close()
        driver.switch_to_window(driver.window_handles[0])
        driver.back()
    else:
        # 창 닫기, 검색 창으로 전환
        driver.close()
        driver.switch_to_window(driver.window_handles[0])
        driver.back()

In [ ]:
### 2. txt 다운로드
# 2-1. 도시계획 조례 txt 다운로드 (전지역)
def getTxt(sido):
    sido = sido
    
    # 시도 선택
    element = driver.find_element_by_id('ctpvCd')
    element.send_keys(sido)
    time.sleep(1)
    
    # 시군구 > 본청 선택
    element = driver.find_element_by_id('sggCd')
    element.send_keys('본청')

    # 법규구분 > 조례 선택
    button = driver.find_element_by_id('chk02') # 조례 선택
    driver.execute_script('arguments[0].click();', button) # 클릭
    time.sleep(1)

    # 검색어 입력
    searchByName = driver.find_element_by_xpath('//*[@id="edtSrchKwd"]')
    time.sleep(1)
    searchByName.clear()
    searchByName.send_keys('도시계획 조례')
    search_click = driver.find_element_by_xpath('//*[@id="btnSrch"]').click()
    
    # 도시계획 조례 클릭
    driver.find_element_by_xpath('//*[@id="srchForm"]/div[2]/div/div[4]/table/tbody/tr[1]/td[2]/a').click()
    time.sleep(1)

    # txt 저장 버튼 클릭
    driver.find_element_by_xpath('//*[@id="btnSave"]').click()
    time.sleep(2)
    driver.switch_to_window(driver.window_handles[1]) # 새창 전환
    driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[1]/div/span/label').click() # 전체선택 해제
    driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[2]/div[1]/span/label').click() # 전체선택 해제
    driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[3]/div/span/label').click() # 부칙선택 해제

    # 목록 이름에 용적률, 건폐율이 포함된 조항만 선택
    clickChecktBox()
    
    # 파일 다운로드
    downloadFile()
    
    # 각 시군구 파일 다운로드
    if len(checked) == 0:
        getTxtSep(sido)
    else:
        pass

# 2-2. 도시계획 조례 txt 다운로드 (각 시군구 별)
def getTxtSep(sido):
    getSigungu(sido)
    for i in sigungu_names:
        # 시군구 > 시군구 선택
        element = driver.find_element_by_id('sggCd')
        element.send_keys(i)
        time.sleep(1)

        # 법규구분 > 조례 선택
        button = driver.find_element_by_id('chk02')
        driver.execute_script('arguments[0].click();', button)
        time.sleep(1)

        # 검색어 입력
        searchByName = driver.find_element_by_xpath('//*[@id="edtSrchKwd"]')
        time.sleep(1)
        searchByName.clear()
        searchByName.send_keys('도시계획 조례')
        search_click = driver.find_element_by_xpath('//*[@id="btnSrch"]').click()

        # 도시계획 조례 클릭
        try:
            driver.find_element_by_xpath('//*[@id="srchForm"]/div[2]/div/div[4]/table/tbody/tr[1]/td[2]/a').click()
            time.sleep(1)

            # txt 저장 버튼 클릭
            driver.find_element_by_xpath('//*[@id="btnSave"]').click()
            time.sleep(2)
            driver.switch_to_window(driver.window_handles[1]) # 새창 전환
            driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[1]/div/span/label').click() # 전체선택 해제
            driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[2]/div[1]/span/label').click() # 전체선택 해제
            driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[3]/div/span/label').click() # 부칙선택 해제

            # 목록 이름에 용적률, 건폐율이 포함된 조항만 선택
            clickChecktBox()

            # 파일 다운로드
            downloadFile()
        except:
            pass

In [ ]:
### 3. 전 지역 도시계획 조례 txt 다운로드 실행
driver = webdriver.Chrome('./driver/chromedriver.exe')
driver.get('https://www.elis.go.kr/allalr/allAlrList')
driver.maximize_window()
getSido()

for sido in tqdm(sido_names):
    getTxt(sido)

driver.close()

### DB 연동

In [ ]:
###################################################################################
import psycopg2

# DB Connect
conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

###################################################################################